In [1]:
import pandas as pd


### Задание 1
#### Создайте датафрейм authors со столбцами author_id и author_name, в которых соответственно содержатся данные: [1, 2, 3] и ['Тургенев', 'Чехов', 'Островский'].
#### Затем создайте датафрейм book cо столбцами author_id, book_title и price, в которых соответственно содержатся данные:
#### [1, 1, 1, 2, 2, 3, 3],
#### ['Отцы и дети', 'Рудин', 'Дворянское гнездо', 'Толстый и тонкий', 'Дама с собачкой', 'Гроза', 'Таланты и поклонники'],
#### [450, 300, 350, 500, 450, 370, 290].

In [2]:
authors = pd.DataFrame({"author_id": [1, 2, 3], "author_name": ['Тургенев', 'Чехов', 'Островский']},
                       columns=['author_id', 'author_name'])
book = pd.DataFrame({"author_id": [1, 1, 1, 2, 2, 3, 3],
                     "book_title": ['Отцы и дети', 'Рудин', 'Дворянское гнездо', 'Толстый и тонкий', 'Дама с собачкой',
                                    'Гроза', 'Таланты и поклонники'],
                     "price": [450, 300, 350, 500, 450, 370, 290]},
                    columns=['author_id', 'book_title', 'price'])
book

,author_id,book_title,price
0,1,Отцы и дети,450
1,1,Рудин,300
2,1,Дворянское гнездо,350
3,2,Толстый и тонкий,500
4,2,Дама с собачкой,450
5,3,Гроза,370
6,3,Таланты и поклонники,290


### Задание 2
#### Получите датафрейм *authors_price*, соединив датафреймы *authors* и *books* по полю *author_id*.

In [3]:
authors_price = pd.merge(authors, book, on="author_id", how="inner")
authors_price

,author_id,author_name,book_title,price
0,1,Тургенев,Отцы и дети,450
1,1,Тургенев,Рудин,300
2,1,Тургенев,Дворянское гнездо,350
3,2,Чехов,Толстый и тонкий,500
4,2,Чехов,Дама с собачкой,450
5,3,Островский,Гроза,370
6,3,Островский,Таланты и поклонники,290


### Задание 3
#### Создайте датафрейм top5, в котором содержатся строки из authors_price с пятью самыми дорогими книгами.

In [4]:
top5 = authors_price.nlargest(5, "price")
top5

,author_id,author_name,book_title,price
3,2,Чехов,Толстый и тонкий,500
0,1,Тургенев,Отцы и дети,450
4,2,Чехов,Дама с собачкой,450
5,3,Островский,Гроза,370
2,1,Тургенев,Дворянское гнездо,350


### Задание 4
#### Создайте датафрейм authors_stat на основе информации из authors_price. В датафрейме authors_stat должны быть четыре столбца:
#### author_name, min_price, max_price и mean_price,
#### в которых должны содержаться соответственно имя автора, минимальная, максимальная и средняя цена на книги этого автора.

In [5]:
authors_stat = authors_price.groupby('author_name').agg(min_price=('price', 'min'),
                                                        max_price=('price', 'max'),
                                                        mean_price=('price', 'mean'))
authors_stat.reset_index()

,author_name,min_price,max_price,mean_price
0,Островский,290,370,330.000000
1,Тургенев,300,450,366.666667
2,Чехов,450,500,475.000000


### Задание 5
#### Создайте новый столбец в датафрейме authors_price под названием cover, в нем будут располагаться данные о том, какая обложка у данной книги - твердая или мягкая. В этот столбец поместите данные из следующего списка:
#### ['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая'].

In [6]:
authors_price = pd.merge(authors_price,
                         pd.DataFrame(['твердая', 'мягкая', 'мягкая', 'твердая', 'твердая', 'мягкая', 'мягкая'],
                                      columns=['cover']),
                         left_index=True, right_index=True)
authors_price

,author_id,author_name,book_title,price,cover
0,1,Тургенев,Отцы и дети,450,твердая
1,1,Тургенев,Рудин,300,мягкая
2,1,Тургенев,Дворянское гнездо,350,мягкая
3,2,Чехов,Толстый и тонкий,500,твердая
4,2,Чехов,Дама с собачкой,450,твердая
5,3,Островский,Гроза,370,мягкая
6,3,Островский,Таланты и поклонники,290,мягкая


#### Для каждого автора посчитайте суммарную стоимость книг в твердой и мягкой обложке. Используйте для этого функцию pd.pivot_table. При этом столбцы должны называться "твердая" и "мягкая", а индексами должны быть фамилии авторов. Пропущенные значения стоимостей заполните нулями, при необходимости загрузите библиотеку Numpy.

In [7]:
book_info = pd.pivot_table(authors_price, index=['author_name'], columns='cover', fill_value=0,
                            aggfunc={'price': 'sum'} )
book_info.reset_index()
book_info

price        
cover       мягкая твердая
author_name               
Островский     660       0
Тургенев       650     450
Чехов            0     950

#### Назовите полученный датасет book_info и сохраните его в формат pickle под названием "book_info.pkl". Затем загрузите из этого файла датафрейм и назовите его book_info2. Удостоверьтесь, что датафреймы book_info и book_info2 идентичны.

In [8]:
book_info.to_pickle('book_info.pkl')
book_info2 = pd.read_pickle('book_info.pkl')
book_info.eq(book_info2).all(axis=1)

author_name
Островский    True
Тургенев      True
Чехов         True
dtype: bool